In [ ]:
! pip install lamoom_cicd
! pip install openai
! pip install pandas
! pip install python-dotenv

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
import pandas as pd

In [ ]:
from lamoom import Prompt


agent = Prompt("answer_on_medical_question")

agent.add("""Answer on medical question of the user:
{question}
          
Please first think out loud before answering.
""", role="system")




In [ ]:
import os
from lamoom_cicd import TestLLMResponsePipe
openai_key = os.getenv("OPENAI_API_KEY")
claude_api_key = os.getenv("CLAUDE_API_KEY")
lamoom_pipe = TestLLMResponsePipe(openai_key=openai_key)
index = 0
from lamoom import Lamoom, AIModelsBehaviour, AttemptToCall, OpenAIModel, C_128K, ClaudeAIModel
client = Lamoom(openai_key=openai_key, claude_key=claude_api_key)

In [ ]:
from lamoom.response_parsers.response_parser import get_json_from_response

gpt4o = AIModelsBehaviour(
    attempts=[
        AttemptToCall(
            ai_model=OpenAIModel(
                model="gpt-4o",
                max_tokens=C_128K
            ),
            weight=100,
        ),
    ]
)

from lamoom_cicd import TestLLMResponsePipe
lamoom_pipe = TestLLMResponsePipe()
results = []

In [ ]:
data_df = pd.read_csv('./test_data/medical_questions_answers.csv')

In [ ]:
for index, row in data_df.iterrows():
    if index > 9:
        continue
    print(f"Processing row {index}")
    question = row['question']
    ideal_answer = row['answer']
    try:
        context = {
            'question': question,
            'ideal_answer': ideal_answer,
        }
        response = client.call(
            agent.id, context, claude, 
            test_data={
                'ideal_answer': ideal_answer
            }
        )
        test_result = lamoom_pipe.compare(ideal_answer, response.content)
        lamoom_pipe.accumulated_results.append(test_result)
        results.append(test_result)
    except Exception as e:
        print(e)

In [ ]:
lamoom_pipe.visualize_test_results()

In [ ]:
for result in results:
    print('\n'.join([str(q.to_dict()) for q in result.questions]))
    print(result.questions[0].ideal_answer)
    print('\n\n')

## Claude

In [ ]:
! pip uninstall anthropic --yes
! pip install anthropic

Found existing installation: anthropic 0.31.2
Uninstalling anthropic-0.31.2:
  Would remove:
    /Users/kateyanchenka/projects/lamoom-cicd/.venv/lib/python3.13/site-packages/anthropic-0.31.2.dist-info/*
    /Users/kateyanchenka/projects/lamoom-cicd/.venv/lib/python3.13/site-packages/anthropic/*
Proceed (Y/n)? 

In [100]:
import dotenv
dotenv.load_dotenv()
import anthropic

client = anthropic.Anthropic()

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
claude = AIModelsBehaviour(
    attempts=[
            AttemptToCall(
                ai_model=ClaudeAIModel(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=4096                
                ),
                weight=100,
            ),
    ]
)


In [ ]:
claude_lamoom_pipe = TestLLMResponsePipe()
results = []

for index, row in data_df.iterrows():
    if index > 9:
        break
    print(f"Processing row {index}")
    question = row['question']
    ideal_answer = row['answer']
    try:
        context = {
            'question': question,
            'ideal_answer': ideal_answer,
        }
        response = client.call(
            agent.id, context, claude, 
            test_data={
                'ideal_answer': ideal_answer
            }
        )
        test_result = claude_lamoom_pipe.compare(ideal_answer, response.content)
        claude_lamoom_pipe.accumulated_results.append(test_result)
        results.append(test_result)
    except Exception as e:
        print(e)

In [ ]:
claude_lamoom_pipe.visualize_test_results()